In [ ]:
from my_functions import *

## Connect to DynamoDB

Initially, we connect to our local database we started as "dynamodb" container:

In [ ]:
set_db()

In [ ]:
from my_functions import client

## Create Table

Let's setup the table with our attributes and indexes:

In [ ]:
delete_app_table()
create_app_table()

## Helpers

To setup some other keys later on, we create a function to extract the id from a key:

In [ ]:
parse_id_from_key("CHAN#7")

## Users

We create and save our users:

In [ ]:
uuid_entry1 = str(uuid.uuid4())
user_martin = {
    'PK': {'S': f"USER#{uuid_entry1}"},
    'SK': {'S': f"USER#{uuid_entry1}"},
    'Name': {'S': 'Martin Marsal'},
}

uuid_entry2 = str(uuid.uuid4())
user_christian = {
    'PK': {'S': f"USER#{uuid_entry2}"},
    'SK': {'S': f"USER#{uuid_entry2}"},
    'Name': {'S': 'Christian Diegmann'},
}

uuid_entry3 = str(uuid.uuid4())
user_robin = {
    'PK': {'S': f"USER#{uuid_entry3}"},
    'SK': {'S': f"USER#{uuid_entry3}"},
    'Name': {'S': 'Robin Schüle'},
}

In [ ]:
save_user(user_martin)
find_user(user_martin['PK']['S'], user_martin['SK']['S'])

In [ ]:
save_user(user_christian)
find_user(user_christian['PK']['S'], user_christian['SK']['S'])

In [ ]:
save_user(user_robin)
find_user(user_robin['PK']['S'], user_robin['SK']['S'])

In [ ]:
add_to_followers(user_martin['PK']['S'], user_christian)

In [ ]:
add_to_followers(user_christian['PK']['S'], user_robin)

In [ ]:
add_to_followers(user_robin['PK']['S'], user_martin)

## 1st access pattern: Post a tweet

In [ ]:
uuid_entry4 = str(uuid.uuid4())
timestamp = int(time.time())
tweet = {'id': {'S': f"{uuid_entry4}"}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
uuid_entry5 = str(uuid.uuid4())
timestamp = int(time.time())
tweet_2 = {'id': {'S': f"{uuid_entry5}"}, 'text': {'S': 'MongoDB ist super!'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
newMartin = find_user(user_martin['PK']['S'], user_martin['SK']['S'])

In [ ]:
post_tweet(user_martin['PK']['S'], tweet)

In [ ]:
post_tweet(user_martin['PK']['S'], tweet_2)

In [ ]:
class TestPostTweet(unittest.TestCase):
    def test_post_tweet_enriched_timeline(self):
        christian_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(christian_timeline['Items'])
        self.assertEqual(timeline_length, 2)

    def test_post_tweet_empty_timeline(self):
        robin_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_robin['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(robin_timeline['Items'])
        self.assertEqual(timeline_length, 0)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

## 2nd access pattern: Write a reply

In [ ]:
uuid_entry6 = str(uuid.uuid4())
reply = {'id': {'S': f"{uuid_entry6}"}, 'text': {'S': 'Hallo zurück.'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}, 'tweetId': {'S': f"{parse_id_from_key(tweet['id']['S'])}"},}

In [ ]:
post_reply(user_martin, user_christian, tweet, reply)

In [ ]:
class TestPostReply(unittest.TestCase):
    def test_post_reply_enriched_replies(self):
        test_tweet = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': f"TIMELINE#{tweet['id']['S']}" }
            },
        )
        tweet_length = len(test_tweet['Items'])
        self.assertEqual(tweet_length, 2)
        
    def test_post_reply_empty_replies(self):
        test_tweet = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': f"TIMELINE#{tweet_2['id']['S']}" }
            },
        )
        tweet_length = len(test_tweet['Items'])
        self.assertEqual(tweet_length, 1)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostReply)
    unittest.TextTestRunner(verbosity=2).run(suite)

# 3rd access pattern: Edit a tweet

In [ ]:
new_tweet_text = "DynamoDB ist super"
edit_tweet(user_martin['PK']['S'], uuid_entry5, new_tweet_text)

In [ ]:
user_martin_tweets = get_user_tweets(user_martin['PK']['S'])
print(user_martin_tweets)

In [ ]:
class TestEditTweet(unittest.TestCase):
    def test_edit_tweet__timeline(self):
        tweet_id_to_find = uuid_entry5  
        updated_text = "DynamoDB ist super"

        christian_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND SK = :sk',
            ExpressionAttributeValues={
                ':pk': {'S': f"USER#{parse_id_from_key(user_christian['PK']['S'])}"},
                ':sk': {'S': f"TIMELINE#{tweet_id_to_find}"}
            },
        )
        matching_items = christian_timeline.get('Items', [])

        self.assertTrue(
            any('text' in item and item['text']['S'] == updated_text for item in matching_items),
            f"Tweet with ID {tweet_id_to_find} not found in timeline or text not updated."
        )

    def test_edit_tweet_martin(self):
        tweet_id_to_find = uuid_entry5  
        updated_text = "DynamoDB ist super"  

        
        martin_tweets = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND SK = :sk',
            ExpressionAttributeValues={
                ':pk': {'S': f"USER#{parse_id_from_key(user_martin['PK']['S'])}"},
                ':sk': {'S': f"TWEET#{tweet_id_to_find}"}
            },
        )
        matching_items_martin = martin_tweets.get('Items', [])

        # Check if the updated text is found in user_martin's tweets
        self.assertTrue(
            any('text' in item and item['text']['S'] == updated_text for item in matching_items_martin),
            f"Tweet with ID {tweet_id_to_find} not found in Martin's tweets or text not updated."
        )


    def test_edit_tweet_empty_timeline(self):
        robin_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_robin['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(robin_timeline['Items'])
        self.assertEqual(timeline_length, 0)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestEditTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

# 4th access pattern: Read a timeline

In [ ]:
user_christian_timeline = read_timeline(user_christian['PK']['S'])
print(user_christian_timeline)

In [ ]:
class TestReadTimeline(unittest.TestCase):
    def test_read_timeline(self):
        # Assuming user_christian has a populated timeline
        expected_texts = ["Moin, moin.", "Hallo zurück.", "DynamoDB ist super"]

        # Fetch the timeline of user_christian
        christian_timeline = read_timeline(user_christian['PK']['S'])

        # Extract text items from the timeline
        actual_texts = [item['text']['S'] for item in christian_timeline]

        # Check if the actual text items match the expected texts (order doesn't matter)
        self.assertCountEqual(actual_texts, expected_texts, "Timeline text items do not match the expected texts.")

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestReadTimeline)
    unittest.TextTestRunner(verbosity=2).run(suite)

# 5th access pattern: Delete a Tweet

In [ ]:
class TestDeleteTweet(unittest.TestCase):
    def test_delete_tweet(self):
        user_id = uuid_entry1
        tweet_id = uuid_entry5
        user_key = f'USER#{user_id}'
        tweet_key = f'TWEET#{tweet_id}'

        delete_tweet(user_id, tweet_id)

        tweet_item = client.get_item(TableName='twitter', Key={'PK': {'S': tweet_key}, 'SK': {'S': tweet_key}})
        self.assertNotIn('Item', tweet_item)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestDeleteTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

# 6th access pattern: Delete a User

In [ ]:
class TestDeleteUser(unittest.TestCase):
    def test_user_deleted(self):
        user_id = uuid_entry1
        user_key = f'USER#{user_id}'

        # Funktion aufrufen
        delete_user(user_id)

        # Überprüfen, ob der Benutzer und der Tweet erfolgreich gelöscht wurden
        user_item = client.get_item(TableName='twitter', Key={'PK': {'S': user_key}, 'SK': {'S': user_key}})
        self.assertNotIn('Item', user_item)

    def test_tweets_deleted(self):
        tweet_id = uuid_entry4
        tweet_key = f'TWEET#{tweet_id}'
        
        tweet_item = client.get_item(TableName='twitter', Key={'PK': {'S': tweet_key}, 'SK': {'S': tweet_key}})
        self.assertNotIn('Item', tweet_item)

In [ ]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestDeleteUser)
    unittest.TextTestRunner(verbosity=2).run(suite)